<a href="https://colab.research.google.com/github/getmubarak/ML/blob/master/Algorithms/keras/LSTM_shakespeare_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time


path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
path_to_file

1122304/1115394 [==============================] - 0s 0us/step


'/root/.keras/datasets/shakespeare.txt'

In [0]:
SHAKESPEARE_TXT = '/root/.keras/datasets/shakespeare.txt'

def transform(txt):
  return np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)

def input_fn(seq_len=1, batch_size=1024):
  """Return a dataset of source and target sequences for training."""
  with tf.io.gfile.GFile(SHAKESPEARE_TXT, 'r') as f:
    txt = f.read()

  source = tf.constant(transform(txt), dtype=tf.int32)

  ds = tf.data.Dataset.from_tensor_slices(source).batch(seq_len+1, drop_remainder=True)

  def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

  BUFFER_SIZE = 10000
  ds = ds.map(split_input_target).shuffle(BUFFER_SIZE).batch(batch_size, drop_remainder=True)

  return ds.repeat()

In [0]:
#Build the model

EMBEDDING_DIM = 512

def lstm_model(seq_len=100, batch_size=None, stateful=True):
  """Language model: predict the next word given the current word."""
  source = tf.keras.Input(
      name='seed', shape=(seq_len,), batch_size=batch_size, dtype=tf.int32)

  #Because our vocabulary size is 256, the input dimension to the Embedding layer is 256. 
  embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
  lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(embedding)
  lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(lstm_1)
  predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='softmax'))(lstm_2)
  return tf.keras.Model(inputs=[source], outputs=[predicted_char])

In [0]:
#train the model

tf.keras.backend.clear_session()


training_model = lstm_model(seq_len=1, stateful=False)
training_model.compile(
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])


In [22]:


training_model.fit(
    input_fn(),
    steps_per_epoch=100,
    epochs=1
)
training_model.save_weights('/tmp/bard.h5', overwrite=True)

100/100 [==============================] - 61s 612ms/step - loss: 2.4499 - sparse_categorical_accuracy: 0.0797


In [23]:
#Make predictions with the model

BATCH_SIZE = 5
PREDICT_LEN = 250

# Keras requires the batch size be specified ahead of time for stateful models.
# We use a sequence length of 1, as we will be feeding in one character at a 
# time and predicting the next character.
prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)
prediction_model.load_weights('/tmp/bard.h5')

# We seed the model with our initial string, copied BATCH_SIZE times

seed_txt = 'Looks it not like the king?  Verily, we must go! '
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)

# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
  prediction_model.predict(seed[:, i:i + 1])

# Now we can accumulate predictions!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
  last_word = predictions[-1]
  next_probits = prediction_model.predict(last_word)[:, 0, :]
  
  # sample from our output distribution
  next_idx = [
      np.random.choice(256, p=next_probits[i])
      for i in range(BATCH_SIZE)
  ]
  predictions.append(np.asarray(next_idx, dtype=np.int32))
  

for i in range(BATCH_SIZE):
  print('PREDICTION %d\n\n' % i)
  p = [predictions[j][i] for j in range(PREDICT_LEN)]
  generated = ''.join([chr(c) for c in p])  # Convert back to text
  print(generated)
  print()
  assert len(generated) == PREDICT_LEN, 'Generated text too short'

PREDICTION 0


 br le g llld'so, vitheley; scee hadintheikeed io br s?
I teery fo cree tor p, tse, g ret.




INENGoubl thilol sod,

Har gr n be hee my h ithe
t wes at heean ges th; de I br lor tol f or ade wis wat and yom, hice cowonome zil,
iair mont asoncueve se

PREDICTION 1


 deso th kist fre asssh co we ntol bistag t d ue thy of, tyo ie lo hide d m t leey th he, m, minge we hiwimove cord aw t ams h mibrarnt t ame yout lody tie stheand g itly abeves made earinerd hamsotonope os ked oe imaitiren, th he 'sthay se pun urite

PREDICTION 2


 umyoathaure spagely treal dowol,'seae hesord w we wn is meespl thopefer. wis d yoounere tilse l ide igs thov h gey atow rf l ne Mu ntey, bey to ds onrit anste 's odle, twin kilodif we walabre ald lar, t y's wan brpre toto k m.
A is, s be ur chow.




PREDICTION 3


 w t It t hibe tcee d ma'do we bamee. faf t pealikit brion-ve reatathe mor ugidiror a m'tomo,

Whean sised thesshel, meak gen toreo oor n 'tieis ts som t at, m my, lid, osse forfe r'ai